In [1]:
import os
print("Current working directory:", os.getcwd())
print("Files here:", os.listdir(".")[:20])

Current working directory: C:\Users\jacks\Downloads\retail-chatter-research\notebooks
Files here: ['.ipynb_checkpoints', '01_reddit_auth_test.ipynb', '02_db_setup.ipynb']


In [2]:
import os
import sqlite3
from datetime import datetime, timezone

# --- Find project root robustly ---
# This notebook lives in: <root>/notebooks/02_db_setup.ipynb
# So root is one level up from the notebook directory.
NOTEBOOK_DIR = os.path.abspath(os.getcwd())
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, "..")) if NOTEBOOK_DIR.endswith("notebooks") else NOTEBOOK_DIR

DB_DIR = os.path.join(PROJECT_ROOT, "data")
DB_PATH = os.path.join(DB_DIR, "retail_chatter.db")

os.makedirs(DB_DIR, exist_ok=True)

conn = sqlite3.connect(DB_PATH)
conn.execute("PRAGMA journal_mode=WAL;")
conn.execute("PRAGMA foreign_keys = ON;")

schema_sql = """
CREATE TABLE IF NOT EXISTS runs (
  run_id INTEGER PRIMARY KEY AUTOINCREMENT,
  run_ts_utc TEXT NOT NULL,
  window_hours INTEGER NOT NULL,
  notes TEXT
);

CREATE TABLE IF NOT EXISTS run_tickers (
  run_id INTEGER NOT NULL,
  ticker TEXT NOT NULL,
  PRIMARY KEY (run_id, ticker),
  FOREIGN KEY (run_id) REFERENCES runs(run_id) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS posts (
  post_id TEXT PRIMARY KEY,
  ticker TEXT NOT NULL,
  subreddit TEXT NOT NULL,
  title TEXT,
  selftext TEXT,
  author TEXT,
  url TEXT,
  score INTEGER,
  num_comments INTEGER,
  created_utc INTEGER,
  retrieved_utc INTEGER NOT NULL
);

CREATE INDEX IF NOT EXISTS idx_posts_ticker_created ON posts (ticker, created_utc);
CREATE INDEX IF NOT EXISTS idx_posts_subreddit_created ON posts (subreddit, created_utc);

CREATE TABLE IF NOT EXISTS comments (
  comment_id TEXT PRIMARY KEY,
  post_id TEXT NOT NULL,
  ticker TEXT NOT NULL,
  subreddit TEXT NOT NULL,
  parent_id TEXT,
  author TEXT,
  body TEXT,
  score INTEGER,
  created_utc INTEGER,
  retrieved_utc INTEGER NOT NULL,
  depth INTEGER DEFAULT 0,
  FOREIGN KEY (post_id) REFERENCES posts(post_id) ON DELETE CASCADE
);

CREATE INDEX IF NOT EXISTS idx_comments_ticker_created ON comments (ticker, created_utc);
CREATE INDEX IF NOT EXISTS idx_comments_post ON comments (post_id);

CREATE TABLE IF NOT EXISTS summaries (
  run_id INTEGER NOT NULL,
  ticker TEXT NOT NULL,
  narrative TEXT,
  themes_json TEXT,
  stance_json TEXT,
  created_utc INTEGER NOT NULL,
  PRIMARY KEY (run_id, ticker),
  FOREIGN KEY (run_id) REFERENCES runs(run_id) ON DELETE CASCADE
);
"""

conn.executescript(schema_sql)
conn.commit()

tables = [r[0] for r in conn.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")]
print("DB created at:", DB_PATH)
print("Tables:", tables)

conn.close()


DB created at: C:\Users\jacks\Downloads\retail-chatter-research\data\retail_chatter.db
Tables: ['comments', 'posts', 'run_tickers', 'runs', 'sqlite_sequence', 'summaries']


In [3]:
import sqlite3, time
from datetime import datetime, timezone

conn = sqlite3.connect(DB_PATH)
conn.execute("PRAGMA foreign_keys = ON;")

run_ts = datetime.now(timezone.utc).isoformat()
window_hours = 48

conn.execute(
    "INSERT INTO runs (run_ts_utc, window_hours, notes) VALUES (?, ?, ?)",
    (run_ts, window_hours, "schema test")
)
run_id = conn.execute("SELECT last_insert_rowid()").fetchone()[0]

for t in ["TSLA", "NVDA"]:
    conn.execute("INSERT OR IGNORE INTO run_tickers (run_id, ticker) VALUES (?, ?)", (run_id, t))

conn.commit()

print("Inserted run_id:", run_id)
print("Tickers for run:", conn.execute("SELECT ticker FROM run_tickers WHERE run_id=? ORDER BY ticker", (run_id,)).fetchall())

conn.close()


Inserted run_id: 1
Tickers for run: [('NVDA',), ('TSLA',)]
